Let say we predict a vcustomer will churn and  model works with 95% accuracy. By calling the customers who are likely to churn and giving an attractive offer, you are assuming 10% of them will retain and bring monthly $20 per each.

That’s a lot of assumptions. Breaking it down:   
The model’s accuracy is 95%. Is it really? You have trained your model based on last month’s data. The next month, there will be new users, new product features, marketing & brand activities, seasonality and so on. Historical accuracy and real accuracy rarely match in this kind of cases. You can’t come up with a conclusion without a test.   

By looking at the previous campaigns’ results, you are assuming a 10% conversion. It doesn’t guarantee that your new action will have 10% conversion due to the factors above. Moreover, since it is a new group, their reaction is partly unpredictable.   

Finally, if those customers bring $20 monthly today, that doesn’t mean they will bring the same after your new action.


In [3]:
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.cluster import KMeans


In [4]:
import sklearn
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import warnings
warnings.filterwarnings("ignore")

## CREATE DATA    

- **customer_id**: the unique identifier of the customer   
- **segment**: customer’s segment; high-value or low-value   
- **group**: indicates whether the customer is in the test or control group  
- **purchase_count**: # of purchases completed by the customer  

In [5]:
#create hv segment
df_hv = pd.DataFrame()
df_hv['customer_id'] = np.array([count for count in range(20000)])
df_hv['segment'] = np.array(['high-value' for _ in range(20000)])
df_hv['group'] = 'control'
df_hv.loc[df_hv.index<10000,'group'] = 'test' 


In [6]:
df_hv.loc[df_hv.group == 'test', 'purchase_count'] = np.random.poisson(0.6, 10000)
df_hv.loc[df_hv.group == 'control', 'purchase_count'] = np.random.poisson(0.5, 10000)

In [7]:
df_hv.group.value_counts()

control    10000
test       10000
Name: group, dtype: int64

In [8]:
df_hv.head(10)

,customer_id,segment,group,purchase_count
0,0,high-value,test,0.0
1,1,high-value,test,1.0
2,2,high-value,test,0.0
3,3,high-value,test,0.0
4,4,high-value,test,1.0
5,5,high-value,test,0.0
6,6,high-value,test,0.0
7,7,high-value,test,0.0
8,8,high-value,test,0.0
9,9,high-value,test,0.0


In [9]:
df_hv.tail(10)

,customer_id,segment,group,purchase_count
19990,19990,high-value,control,0.0
19991,19991,high-value,control,0.0
19992,19992,high-value,control,0.0
19993,19993,high-value,control,1.0
19994,19994,high-value,control,1.0
19995,19995,high-value,control,1.0
19996,19996,high-value,control,0.0
19997,19997,high-value,control,0.0
19998,19998,high-value,control,1.0
19999,19999,high-value,control,0.0


In [10]:
test_results = df_hv[df_hv.group == 'test'].purchase_count
control_results = df_hv[df_hv.group == 'control'].purchase_count

hist_data = [test_results, control_results]

group_labels = ['test', 'control']

# T-test

In [11]:
from scipy import stats 
test_result = stats.ttest_ind(test_results, control_results)
print(test_result)

Ttest_indResult(statistic=9.382071878483577, pvalue=7.1403800481125e-21)


In [12]:
def eval_test(test_results,control_results):
    test_result = stats.ttest_ind(test_results, control_results)
    if test_result[1] < 0.05:
        print('result is significant')
    else:
        print('result is not significant')
        
eval_test(test_results,control_results)

result is significant


# Blocking

In [13]:
#create hv segment
df_hv = pd.DataFrame()
df_hv['customer_id'] = np.array([count for count in range(20000)])
df_hv['segment'] = np.array(['high-value' for _ in range(20000)])
df_hv['prev_purchase_count'] = np.random.poisson(0.9, 20000)


df_lv = pd.DataFrame()
df_lv['customer_id'] = np.array([count for count in range(20000,100000)])
df_lv['segment'] = np.array(['low-value' for _ in range(80000)])
df_lv['prev_purchase_count'] = np.random.poisson(0.3, 80000)

df_customers = pd.concat([df_hv,df_lv],axis=0)
df_customers.head()

,customer_id,segment,prev_purchase_count
0,0,high-value,0
1,1,high-value,0
2,2,high-value,0
3,3,high-value,0
4,4,high-value,0


In [14]:
df_customers.tail()

,customer_id,segment,prev_purchase_count
79995,99995,low-value,0
79996,99996,low-value,1
79997,99997,low-value,0
79998,99998,low-value,0
79999,99999,low-value,0


In [15]:
len(df_customers)

100000

In [16]:
df_test = df_customers.sample(frac=0.9)
df_control = df_customers[~df_customers.customer_id.isin(df_test.customer_id)]

df_test.segment.value_counts()

low-value     71981
high-value    18019
Name: segment, dtype: int64

In [17]:
df_control.segment.value_counts()

low-value     8019
high-value    1981
Name: segment, dtype: int64

In [18]:
df_test_hv = df_customers[df_customers.segment == 'high-value'].sample(frac=0.9)
df_test_lv = df_customers[df_customers.segment == 'low-value'].sample(frac=0.9)

df_test = pd.concat([df_test_hv,df_test_lv],axis=0)
df_control = df_customers[~df_customers.customer_id.isin(df_test.customer_id)]

In [19]:
df_test.segment.value_counts()

low-value     72000
high-value    18000
Name: segment, dtype: int64

In [20]:
df_control.segment.value_counts()

low-value     8000
high-value    2000
Name: segment, dtype: int64

# One-way Anova

In [21]:
#create hv segment
df_hv = pd.DataFrame()
df_hv['customer_id'] = np.array([count for count in range(30000)])
df_hv['segment'] = np.array(['high-value' for _ in range(30000)])
df_hv['group'] = 'A'
df_hv.loc[df_hv.index>=10000,'group'] = 'B' 
df_hv.loc[df_hv.index>=20000,'group'] = 'C' 


In [22]:
df_hv.group.value_counts()

B    10000
A    10000
C    10000
Name: group, dtype: int64

In [23]:
df_hv.loc[df_hv.group == 'A', 'purchase_count'] = np.random.poisson(0.4, 10000)
df_hv.loc[df_hv.group == 'B', 'purchase_count'] = np.random.poisson(0.6, 10000)
df_hv.loc[df_hv.group == 'C', 'purchase_count'] = np.random.poisson(0.2, 10000)

In [24]:
a_stats = df_hv[df_hv.group=='A'].purchase_count
b_stats = df_hv[df_hv.group=='B'].purchase_count
c_stats = df_hv[df_hv.group=='C'].purchase_count

hist_data = [a_stats, b_stats, c_stats]

group_labels = ['A', 'B','C']

In [25]:
def one_anova_test(a_stats,b_stats,c_stats):
    test_result = stats.f_oneway(a_stats, b_stats, c_stats)
    if test_result[1] < 0.05:
        print('result is significant')
    else:
        print('result is not significant')
        

In [26]:
one_anova_test(a_stats,b_stats,c_stats)

result is significant


In [27]:
df_hv.loc[df_hv.group == 'A', 'purchase_count'] = np.random.poisson(0.5, 10000)
df_hv.loc[df_hv.group == 'B', 'purchase_count'] = np.random.poisson(0.5, 10000)
df_hv.loc[df_hv.group == 'C', 'purchase_count'] = np.random.poisson(0.5, 10000)

In [28]:
a_stats = df_hv[df_hv.group=='A'].purchase_count
b_stats = df_hv[df_hv.group=='B'].purchase_count
c_stats = df_hv[df_hv.group=='C'].purchase_count

hist_data = [a_stats, b_stats, c_stats]

group_labels = ['A', 'B','C']

In [29]:
one_anova_test(a_stats,b_stats,c_stats)

result is not significant


# Two-way Anova

In [30]:
#create hv segment
df_hv = pd.DataFrame()
df_hv['customer_id'] = np.array([count for count in range(20000)])
df_hv['segment'] = np.array(['high-value' for _ in range(20000)])
df_hv['group'] = 'control'
df_hv.loc[df_hv.index<10000,'group'] = 'test' 
df_hv.loc[df_hv.group == 'control', 'purchase_count'] = np.random.poisson(0.6, 10000)
df_hv.loc[df_hv.group == 'test', 'purchase_count'] = np.random.poisson(0.8, 10000)


df_lv = pd.DataFrame()
df_lv['customer_id'] = np.array([count for count in range(20000,100000)])
df_lv['segment'] = np.array(['low-value' for _ in range(80000)])
df_lv['group'] = 'control'
df_lv.loc[df_lv.index<40000,'group'] = 'test' 
df_lv.loc[df_lv.group == 'control', 'purchase_count'] = np.random.poisson(0.2, 40000)
df_lv.loc[df_lv.group == 'test', 'purchase_count'] = np.random.poisson(0.3, 40000)

df_customers = pd.concat([df_hv,df_lv],axis=0)


In [31]:
df_customers.head()

,customer_id,segment,group,purchase_count
0,0,high-value,test,0.0
1,1,high-value,test,0.0
2,2,high-value,test,1.0
3,3,high-value,test,0.0
4,4,high-value,test,0.0


In [32]:
import statsmodels.formula.api as smf 
from statsmodels.stats.anova import anova_lm
model = smf.ols(formula='purchase_count ~ segment + group ', data=df_customers).fit()
aov_table = anova_lm(model, typ=2)

In [33]:
print(np.round(aov_table,4))

              sum_sq       df          F  PR(>F)
segment    3152.9329      1.0  9126.2338     0.0
group       420.2929      1.0  1216.5470     0.0
Residual  34546.9817  99997.0        NaN     NaN


In [34]:
#create hv segment
df_hv = pd.DataFrame()
df_hv['customer_id'] = np.array([count for count in range(20000)])
df_hv['segment'] = np.array(['high-value' for _ in range(20000)])
df_hv['group'] = 'control'
df_hv.loc[df_hv.index<10000,'group'] = 'test' 
df_hv.loc[df_hv.group == 'control', 'purchase_count'] = np.random.poisson(0.8, 10000)
df_hv.loc[df_hv.group == 'test', 'purchase_count'] = np.random.poisson(0.8, 10000)


df_lv = pd.DataFrame()
df_lv['customer_id'] = np.array([count for count in range(20000,100000)])
df_lv['segment'] = np.array(['low-value' for _ in range(80000)])
df_lv['group'] = 'control'
df_lv.loc[df_lv.index<40000,'group'] = 'test' 
df_lv.loc[df_lv.group == 'control', 'purchase_count'] = np.random.poisson(0.2, 40000)
df_lv.loc[df_lv.group == 'test', 'purchase_count'] = np.random.poisson(0.2, 40000)

df_customers = pd.concat([df_hv,df_lv],axis=0)


In [35]:
import statsmodels.formula.api as smf 
from statsmodels.stats.anova import anova_lm
model = smf.ols(formula='purchase_count ~ segment + group ', data=df_customers).fit()
aov_table = anova_lm(model, typ=2)

In [36]:
print(np.round(aov_table,4))

              sum_sq       df           F  PR(>F)
segment    5830.0517      1.0  18173.7981  0.0000
group         0.3062      1.0      0.9547  0.3285
Residual  32078.4724  99997.0         NaN     NaN


# Calculation sample size

In [37]:
from statsmodels.stats import power
ss_analysis = power.TTestIndPower()

In [38]:
#create hv segment
df_hv = pd.DataFrame()
df_hv['customer_id'] = np.array([count for count in range(20000)])
df_hv['segment'] = np.array(['high-value' for _ in range(20000)])
df_hv['prev_purchase_count'] = np.random.poisson(0.7, 20000)


In [39]:
df_hv.head()

,customer_id,segment,prev_purchase_count
0,0,high-value,1
1,1,high-value,2
2,2,high-value,1
3,3,high-value,1
4,4,high-value,0


In [40]:
purchase_mean = df_hv.prev_purchase_count.mean()
purchase_std = df_hv.prev_purchase_count.std()

In [41]:
print(np.round(purchase_mean,4),np.round(purchase_std,4))

0.6936 0.8302


In [42]:
effect_size = (0.75 - purchase_mean)/purchase_std

In [43]:
alpha = 0.05
power = 0.8
ratio = 1

In [44]:
ss_result = ss_analysis.solve_power(effect_size=effect_size, power=power,alpha=alpha, ratio=ratio , nobs1=None) 
print(ss_result)

3408.3012684116857


In [45]:
def calculate_sample_size(c_data, column_name, target,ratio):
    value_mean = c_data[column_name].mean()
    value_std = c_data[column_name].std()
    
    value_target = value_mean * target
    
    effect_size = (value_target - value_mean)/value_std
    
    power = 0.8
    alpha = 0.05
    ss_result = ss_analysis.solve_power(effect_size=effect_size, power=power,alpha=alpha, ratio=ratio , nobs1=None) 
    print(int(ss_result))

In [46]:
calculate_sample_size(df_hv, 'prev_purchase_count', 1.05,1)

8995
